In [1]:
import geocoder
import ephem
import ephem.stars
import datetime
import math
import cmath
import sympy
import functools
import numpy as np
import qutip 
import random
import mpmath
import scipy

def xyz_c(x, y, z):
    if z == 1:
        return float('inf') 
    else:
        return complex(x/(1-z), y/(1-z))

def c_xyz(c):
    if c == float('inf'):
        return [0,0,1]
    x = c.real
    y = c.imag
    return [(2*x)/(1.+(x**2)+(y**2)), \
           (2*y)/(1.+(x**2)+(y**2)), \
           (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]

def altaz_xyz(altitude, azimuth):
    x = math.sin(azimuth)*math.cos(altitude)
    y = math.cos(altitude)*math.cos(azimuth)
    z = math.sin(altitude)
    return [x, y, z]

def xyz_spherical(x, y, z):
    r = math.sqrt(x**2 + y**2 + z**2)
    theta = math.acos(z/r)
    phi = math.atan(y/z)
    return [r, theta, phi]

def spherical_xyz(r, theta, phi):
    x = r*math.sin(theta)*math.cos(phi)
    y = r*math.sin(theta)*math.sin(phi)
    z = r*math.cos(theta)
    return [x, y, z]

def spherical_state(r, theta, phi):
    return np.array([math.cos(theta/2), cmath.exp(complex(0,1)*phi)*theta/2])

def spherical_c(altitude, azimuth):
    R, THETA = None, None
    THETA = azimuth
    if altitude == 0:
        R = float('inf')
    else:
        R = math.sin(altitude)/(1-math.cos(altitude))
    return cmath.rect(R, THETA)

def c_spherical(c):
    ALTITUDE, AZIMUTH = None, None
    AZIMUTH = th
    r, th = cmath.polar(c)
    if r == 0:
        ALTITUDE = math.pi
    else:
        ALTITUDE = 2*math.arctan(1/r)
    return [ALTITUDE, AZIMUTH]

def xyz_qubit(x, y, z):
    return (x*qutip.sigmax() + y*qutip.sigmay() + z*qutip.sigmaz()).full()

def txyz_qubit(t, x, y, z):
    return (t*qutip.identity(2) + x*qutip.sigmax(2) + y*qutip.sigmay() + z*qutip.sigmaz()).full()

def scalar_product(m, n):
    return 0.5*np.trace(np.dot(np.conjugate(m).T, n))

def qubit_txyz(m):
    t = scalar_product(np.eye(2), m).real
    x = scalar_product(qutip.sigmax().full(), m).real
    y = scalar_product(qutip.sigmay().full(), m).real
    z = scalar_product(qutip.sigmaz().full(), m).real
    return [t/math.sqrt(t**2+x**2+y**2+z**2), x/t, y/t, z/t]

def distinguish_qubits(state):
    state.dims = [[2,2,2],[1,1,1]]
    traces = [state.ptrace(i).full() for i in range(3)]
    state.dims = [[8],[1]]
    return traces

def upgrade(qubit_measurement, q):
    total_measurement = None
    if q == 0:
        total_measurement = qutip.tensor(qubit_measurement, qutip.identity(2), qutip.identity(2))
    elif q == 1:
        total_measurement = qutip.tensor(qutip.identity(2), qubit_measurement, qutip.identity(2))
    elif q == 2:
        total_measurement = qutip.tensor(qutip.identity(2), qutip.identity(2), qubit_measurement)
    return total_measurement

def roots_polynomial(roots):
    s = sympy.symbols("s")
    polynomial = sympy.Poly(functools.reduce(lambda a, b: a*b, [s+root for root in roots]), domain="CC")
    return polynomial.coeffs()

def polynomial_roots(polynomial):
    try:
        roots = [complex(root) for root in mpmath.polyroots(polynomial)]
    except:
        return [complex(0,0) for i in range(len(polynomial)-1)]
    return roots

def combos(a,b):
    f = math.factorial
    return f(a) / f(b) / f(a-b)

def polynomial_state(polynomial):
    coordinates = [polynomial[i]/(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) for i in range(len(polynomial))]
    return qutip.Qobj(np.array(coordinates).T)

def state_polynomial(v):
    polynomial = v.T.tolist()
    return [(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) * polynomial[i] for i in range(len(polynomial))]

def state_xyz(v):
    return [c_xyz(root) for root in polynomial_roots(state_polynomial(v))]

def sym_xyz(v):
    s = qutip.Qobj(v)
    t = qutip.expect(qutip.identity(2), s)
    x = qutip.expect(qutip.sigmax(), s)
    y = qutip.expect(qutip.sigmay(), s)
    z = qutip.expect(qutip.sigmaz(), s)
    #return [t/math.sqrt(t**2+x**2+y**2+z**2), x/t, y/t, z/t]
    return [x/t, y/t, z/t]

def state_spherical(state):
    return [xyz_spherical(*xyz) for xyz in state_xyz(state)]

def state_states(state):
    return [spherical_state(*spherical) for spherical in state_spherical(state)]

def states_state(states):
    return polynomial_state(roots_polynomial([xyz_c(*spherical_xyz(*state_spherical(state)[0])) for state in states]))

def make_unitary(h, dt):
    return scipy.linalg.expm(-2*math.pi*complex(0,1)*h*dt)

def A(t):
    return np.array([[np.cosh(t), np.sinh(t)], [np.sinh(t), np.cosh(t)]])

def N(t):
    return np.array([[1, 0], [t, 1]])

def K(t):
    return np.array([[np.cos(t), -1*np.sin(t)],[np.sin(t), np.cos(t)]])

def apply_mobius(m, states):
    return [np.dot(m, state) for state in states]

def qutip_numpy(state):
    return state.full().T[0]